In [17]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from collections import Counter
import re
import nltk
nltk.download('stopwords');
import time
from collections import defaultdict
from array import array
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import math
import collections
from numpy import linalg as la
import string
import pickle
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\xoelr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Load data

In [2]:
with open("tweets.json", "rb") as f:
    data = f.readlines()
    data = [json.loads(str_) for str_ in data]

In [3]:
df_tweets = pd.DataFrame.from_records(data)
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110652 entries, 0 to 110651
Data columns (total 37 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   created_at                 110652 non-null  object 
 1   id                         110652 non-null  int64  
 2   id_str                     110652 non-null  object 
 3   text                       110652 non-null  object 
 4   display_text_range         20849 non-null   object 
 5   source                     110652 non-null  object 
 6   truncated                  110652 non-null  bool   
 7   in_reply_to_status_id      16142 non-null   float64
 8   in_reply_to_status_id_str  16142 non-null   object 
 9   in_reply_to_user_id        17985 non-null   float64
 10  in_reply_to_user_id_str    17985 non-null   object 
 11  in_reply_to_screen_name    17985 non-null   object 
 12  user                       110652 non-null  object 
 13  geo                        18

In [10]:
df_tweets_rts = df_tweets.dropna(subset=['retweeted_status'])
df_tweets_rts = df_tweets_rts.reset_index()

0        {'id': 25127561, 'id_str': '25127561', 'name':...
1        {'id': 237104507, 'id_str': '237104507', 'name...
2        {'id': 1229091405152256001, 'id_str': '1229091...
3        {'id': 822810049, 'id_str': '822810049', 'name...
4        {'id': 169255493, 'id_str': '169255493', 'name...
                               ...                        
54281    {'id': 3254964066, 'id_str': '3254964066', 'na...
54282    {'id': 988825883308838913, 'id_str': '98882588...
54283    {'id': 16260743, 'id_str': '16260743', 'name':...
54284    {'id': 745045295340359680, 'id_str': '74504529...
54285    {'id': 795295891154411521, 'id_str': '79529589...
Name: user, Length: 54286, dtype: object

In [5]:
df_tweets_rts.loc[1,'retweeted_status']

{'created_at': 'Mon Nov 30 02:06:08 +0000 2020',
 'id': 1333230802298052609,
 'id_str': '1333230802298052609',
 'text': 'Again, please take this pandemic seriously. Wear a mask. Wash your hands. Social distance. Limit travel. Follow CDC… https://t.co/oQs1Jg94tW',
 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
 'truncated': True,
 'in_reply_to_status_id': 1333230801337520130,
 'in_reply_to_status_id_str': '1333230801337520130',
 'in_reply_to_user_id': 2617656624,
 'in_reply_to_user_id_str': '2617656624',
 'in_reply_to_screen_name': 'RileyBehrens',
 'user': {'id': 2617656624,
  'id_str': '2617656624',
  'name': 'Riley Behrens',
  'screen_name': 'RileyBehrens',
  'location': 'Phoenix, AZ',
  'url': None,
  'description': '@Harvard | he/him/his | national athlete | politico | mental health advocate | puppy dad',
  'translator_type': 'none',
  'protected': False,
  'verified': False,
  'followers_count': 2321,
  'friends_count': 455,
  'listed_count': 

# Build data for graph

In [13]:
df_graph = pd.DataFrame(columns=["source", "destination"])
df_graph['source'] = [user['screen_name'] for user in df_tweets_rts['user']]
df_graph['destination'] = [retweet['user']['screen_name'] for retweet in df_tweets_rts['retweeted_status']]
df_graph.drop_duplicates(inplace=True)
df_graph.head()

,source,destination
0,TuAnhPhan,RepLeeZeldin
1,Sgt_Martin13,RileyBehrens
2,redianbot,mick_hyde
3,bamynchebbz,F1
4,AndyrMichel,rosscolquhoun


# Build graph

In [35]:
import networkx as nx

In [111]:
G = nx.Graph()
G.add_edges_from(df_graph.values)

In [112]:
_, test = train_test_split(df_graph, test_size=0.2, random_state=42)
test.head()

,source,destination
15325,CarlosRMA94,F1
52766,fenixedu,zeynep
53570,Justloo85230955,MelodyMac51
3001,cheryl_hoekstra,hallbowdes
24999,HamdanNr,likaluca


In [113]:
G_test = nx.Graph()
G_test.add_edges_from(test.values)

In [117]:
def find_nodes_at_distance_2(graph):
    """
    starting from a graph this function returns all the nodes at distance 2
    """
    

    all_potential_recommendations = set()
    
    for n1 in graph.nodes:
        
        # all the nodes at distance 1
        nodes_at_most_distant_1 = set([n for n in graph.neighbors(n1)])
        nodes_at_most_distant_2 = set()
        # all the nodes at distance 1 and distance 2
        for node in nodes_at_most_distant_1:
            for n in graph.neighbors(node):
                nodes_at_most_distant_2.add(n)
        
        
        # only the nodes at distance 2
        only_nodes_at_distance_2 = nodes_at_most_distant_2 - nodes_at_most_distant_1
        
        
        # check if empty set
        if len(only_nodes_at_distance_2) > 0:
            

            for n2 in only_nodes_at_distance_2:
                
                if n1 != n2:
                    all_potential_recommendations.add((n1, n2))
            
    return list(all_potential_recommendations)

In [118]:
potential_recommendations = find_nodes_at_distance_2(G_test)

# Adamic Adar Index

In [131]:

preds = nx.adamic_adar_index(G_test,ebunch = potential_recommendations)

preds = list(preds)
predicted = dict()
for node in G_test.nodes:
    preds_where_node = []
    for i in preds:
        if i[0] == node:
            preds_where_node.append(i)
    best = sorted(preds_where_node, key=lambda tup: tup[2], reverse=True)[:10]
    predicted[node] = best


In [135]:
predicted

{'CarlosRMA94': [('CarlosRMA94', 'leenduati', 0.17155761360662464),
  ('CarlosRMA94', 'KENNo777', 0.17155761360662464),
  ('CarlosRMA94', 'CtlnLucas', 0.17155761360662464),
  ('CarlosRMA94', 'OJiEF1', 0.17155761360662464),
  ('CarlosRMA94', 'ccmndhd', 0.17155761360662464),
  ('CarlosRMA94', 'rosukara', 0.17155761360662464),
  ('CarlosRMA94', 'RacingStorki', 0.17155761360662464),
  ('CarlosRMA94', 'souitirou_51', 0.17155761360662464),
  ('CarlosRMA94', 'walnut_85', 0.17155761360662464),
  ('CarlosRMA94', 'kishiro26', 0.17155761360662464)],
 'F1': [('F1', 'wtf1official', 2.8853900817779268),
  ('F1', 'zerohedge', 1.4426950408889634),
  ('F1', 'sebinred', 1.4426950408889634)],
 'fenixedu': [('fenixedu', 'VictorLGreen1', 0.40242960438184466),
  ('fenixedu', 'genperisi', 0.40242960438184466),
  ('fenixedu', 'alexander_suh', 0.40242960438184466),
  ('fenixedu', 'jbyrneirl', 0.40242960438184466),
  ('fenixedu', 'fpavageau', 0.40242960438184466),
  ('fenixedu', 'timbunce', 0.40242960438184466)

### ALS

In [136]:
from implicit.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix

In [139]:
model = AlternatingLeastSquares(factors=10, calculate_training_loss=True,  iterations=5)

M = nx.linalg.graphmatrix.adjacency_matrix(G_test)
M = csr_matrix(M)

model.fit(M)


In [140]:
model.user_factors

array([[ 1.3738580e-01, -2.4274616e-02,  8.3386004e-02, ...,
         2.1319157e-01, -2.1494561e-01, -1.4020595e-02],
       [ 6.5224624e-01, -1.3892014e+00,  2.2653542e+00, ...,
         3.6839356e+00,  1.3296015e+00, -2.2123203e-01],
       [-1.9528619e-05,  5.8835518e-05,  9.7919299e-05, ...,
        -4.6255809e-06,  2.3775756e-05,  5.6637873e-06],
       ...,
       [-2.7324143e-04,  8.2105782e-04,  4.1950555e-04, ...,
         2.9324141e-04, -1.0725846e-04,  8.3391351e-04],
       [-5.2969006e-07, -4.2864990e-06,  9.3567105e-06, ...,
         4.7399953e-07,  6.8950112e-06, -2.5189283e-06],
       [-2.2043132e-06,  2.2620939e-05,  4.4514058e-05, ...,
         5.3178701e-07,  1.2154109e-05, -1.2327429e-05]], dtype=float32)

In [144]:
predictions = model.recommend_all(M, N = 10)

In [148]:
predictions

array([[  194,  6907, 11903, ...,    34,   334,    70],
       [   40,   156,   773, ...,  5355,  9897, 10771],
       [   60,   773,   199, ...,  8950,  7435,  5445],
       ...,
       [   60,   859,   773, ...,    34,    13,   440],
       [  773,   199,    60, ...,  2212,  3563,   112],
       [  773,    60,   199, ...,     1,  3563,  2212]], dtype=int32)

In [158]:
list(G_test.nodes)

['CarlosRMA94',
 'F1',
 'fenixedu',
 'zeynep',
 'Justloo85230955',
 'MelodyMac51',
 'cheryl_hoekstra',
 'hallbowdes',
 'HamdanNr',
 'likaluca',
 'PenelopePops1',
 'ConnieSchultz',
 'TheReaLizAdam',
 'devisridhar',
 'frommetoj',
 'jaehyunbom__',
 'ChrisRi80029765',
 '2ordinary1',
 'JAEHYUN97FC',
 'SM_NCT',
 'kazunori11',
 'HenrietaDragun1',
 'AngelaRayner',
 'collywolly41',
 'FAustralians',
 'thelittlegiantz',
 'KoreanUpdates',
 'TwitchellJohn',
 'RichardGrenell',
 '0hmyice',
 'OH_mes2',
 'BAMcKinney',
 'GeorgeTakei',
 'hey_beachshell',
 'DrEricDing',
 'Godfrey28482926',
 'ninnyd101',
 'ZeptoSymbolz',
 'justincousson',
 'mikepcooper',
 'ClarkeMicah',
 'Michiganocds',
 'tedcruz',
 'uglyAnubis',
 '_orsteen',
 'Fahad4014',
 'LeAnam__',
 'BabybirdChirps',
 '7aebum',
 'LloydHousley',
 'davidallengreen',
 'purpleelone',
 'nctdaoying',
 'mulieann',
 'ASlavitt',
 'jlleee1',
 'Germanchick1973',
 'si_talopram',
 'andrew_lilico',
 'SebastianHaze',
 'Public_Citizen',
 'Mayades3',
 'theseoulstory',


In [154]:
len(predictions)

13059

In [163]:
right_pred = 0
for i in range(len(predictions)):
    node1 = list(G_test.nodes())[i]
    for node2 in predictions[i]:
        if node1 in G.neighbors(list(G_test.nodes())[node2]):
            right_pred +=1

print('Accuracy = ' + str(right_pred/(len(predictions)*10)))


Accuracy = 0.0025499655410062025


### Page Rank

In [172]:
predictions = dict()
for node in list(G_test.nodes())[:100]:
    best = [k for k, v in sorted(nx.pagerank(G_test,personalization={node:1}).items(), key=lambda item: item[1],reverse=True)][:10]
    predictions[node] = best


In [173]:
predictions

{'CarlosRMA94': ['F1',
  'CarlosRMA94',
  'wtf1official',
  'barbara_nunes__',
  'miduno2k',
  'peteranyadike_',
  'hasrolmd',
  'zerohedge',
  'kazunori11',
  'kyn737'],
 'F1': ['F1',
  'wtf1official',
  'barbara_nunes__',
  'miduno2k',
  'peteranyadike_',
  'hasrolmd',
  'zerohedge',
  'CarlosRMA94',
  'kazunori11',
  'kyn737'],
 'fenixedu': ['zeynep',
  'fenixedu',
  'Em_Lickspittle',
  'jbyrneirl',
  'ChengSoonOng',
  'VictorLGreen1',
  'DomJofo',
  'jankatins',
  'alexander_suh',
  'genperisi'],
 'zeynep': ['zeynep',
  'fenixedu',
  'Em_Lickspittle',
  'jbyrneirl',
  'ChengSoonOng',
  'VictorLGreen1',
  'DomJofo',
  'jankatins',
  'alexander_suh',
  'genperisi'],
 'Justloo85230955': ['Justloo85230955',
  'MelodyMac51',
  'redianbot',
  'drdavidsamadi',
  'ClarkeMicah',
  'thebradfordfile',
  'davidkurten',
  'CNN',
  'DrEricDing',
  'CoronaUpdateBot'],
 'MelodyMac51': ['MelodyMac51',
  'Justloo85230955',
  'redianbot',
  'drdavidsamadi',
  'ClarkeMicah',
  'thebradfordfile',
  'da

### Jaccard

In [174]:
def compute_Jaccard(u,v, graph):
    """
    compute jaccard similarity
    """
    

    # set of neighbors of u
    outlinks_from_u = set(graph.neighbors(u))

    # set of neighbors of v
    inlinks_to_v = set(graph.neighbors(v))

    # intesection of the two sets
    num = outlinks_from_u.intersection(inlinks_to_v)
    
    # union of the two sets
    den = outlinks_from_u.union(inlinks_to_v)
    
    # final division
    out = len(num)/len(den)
    
    return out

In [175]:
predictions = dict()
for node1 in list(G_test.nodes())[:100]:
    similarities = []
    for node2 in list(G_test.nodes())[:100]:
        similarities.append((node2, compute_Jaccard(node1, node2, G_test)))
    similarities = sorted(similarities,reverse=True)[:10]
    
    predictions[node1] = similarities

In [176]:
predictions

{'CarlosRMA94': [('zeynep', 0.0),
  ('uglyAnubis', 0.0),
  ('theseoulstory', 0.0),
  ('therightarticle', 0.0),
  ('thelittlegiantz', 0.0),
  ('tedcruz', 0.0),
  ('si_talopram', 0.0),
  ('shaniben_ezra', 0.0),
  ('rajubanerjeebjp', 0.0),
  ('purpleelone', 0.0)],
 'F1': [('zeynep', 0.0),
  ('uglyAnubis', 0.0),
  ('theseoulstory', 0.0),
  ('therightarticle', 0.0),
  ('thelittlegiantz', 0.0),
  ('tedcruz', 0.0),
  ('si_talopram', 0.0),
  ('shaniben_ezra', 0.0),
  ('rajubanerjeebjp', 0.0),
  ('purpleelone', 0.0)],
 'fenixedu': [('zeynep', 0.0),
  ('uglyAnubis', 0.0),
  ('theseoulstory', 0.0),
  ('therightarticle', 0.0),
  ('thelittlegiantz', 0.0),
  ('tedcruz', 0.0),
  ('si_talopram', 0.0),
  ('shaniben_ezra', 0.0),
  ('rajubanerjeebjp', 0.0),
  ('purpleelone', 0.0)],
 'zeynep': [('zeynep', 1.0),
  ('uglyAnubis', 0.0),
  ('theseoulstory', 0.0),
  ('therightarticle', 0.0),
  ('thelittlegiantz', 0.0),
  ('tedcruz', 0.0),
  ('si_talopram', 0.0),
  ('shaniben_ezra', 0.0),
  ('rajubanerjeebjp', 